<a href="https://colab.research.google.com/github/EnDFLab/EMCS_Project/blob/smart_meter_data_analytics/Smart_Meter_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing Required Packages**

In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pytz

**Creating a Dictionary for Consumers name**

In [31]:
consumer = {"physics":"1DN8M_FNpJ73Zb43nap6910TrStzjIlyF",
              "management":"1DcuVrXwOICXyI_KtcYxp2WWEtsVn7m6x",
              "electrical":"1DnhLgBQ8YSqDmI7rA8nplQEAyOWH_oiO",
              "civil":"1DaW-MB6cVM8jm5MPTg8br6pxJdqIEsEQ",
              "boyshostel":"1DsGE4NfgtfM8CvPQ-Vvh5EjW2MMo1YXW",
              "biotech":"1D_Bb5EJHCejxnhzdhEqc6geRXxWtGr4_",
              "transformer":"1DynnA5YDAH9xJq7KIhIc4fSzrvH_y_22"
              }

**Function to create a df from a csv file and remove unwanted rows, column.**

In [32]:
def read_raw_csv(id):
  if id in consumer.keys():
    Url = "https://drive.google.com/uc?export=download&id=" + consumer[id]
    df = pd.read_csv(Url,skiprows =1)
    df.columns = df.columns.str.strip() # Remove Leading and Trailing Whitespaces from Column
    df.set_index("Last Updated",inplace = True);
    df.drop(labels=["Voltage(V)","Current(A)","Power(W)","PF","Exported Energy(kWh)"],axis=1,inplace=True)
    return(df)
  else:
    return(None)


**Functions for Data Preprocessing**

In [33]:
def data_preprocessing(df):
      df = replace_comma(df)
      #df.to_csv("/content/drive/MyDrive/sorted_data/" + "physics_comma_replaced.csv",float_format='%.2f');
      df = df_total(df)
      #df.to_csv("/content/drive/MyDrive/sorted_data/" + "physics_time_stamp_added.csv",float_format='%.2f');
      df = fill_missing_timestamp(df)
      #df.to_csv("/content/drive/MyDrive/sorted_data/" + "physics_added_time_stamp_added.csv",float_format='%.2f');
      df = fill_missing_value(df)
      #df.to_csv("/content/drive/MyDrive/sorted_data/" + "physics_added_missing_value_added.csv",float_format='%.2f');
      df = calculate_consumption(df)
      #df.to_csv("/content/drive/MyDrive/sorted_data/" + "physics_consumption.csv",float_format='%.2f');
      return df

**Function to Calculate Actual Energy Consumption from unit consumed**

In [34]:
def calculate_consumption(df):
  df["Load_consumed (kWh)"] = df["Load_consumed (kWh)"] - df["Load_consumed (kWh)"].shift(1)
  df.drop(index=df.index[0],axis=0,inplace=True)
  return df

**Function to fill missing values of energy consumption**

In [35]:
def fill_missing_value(df):
    """
    Fill missing values in the 'Load_consumed (kWh)' column using linear interpolation.

    Args:
        df (pd.DataFrame): Input DataFrame with a 'Load_consumed (kWh)' column.

    Returns:
        pd.DataFrame: DataFrame with missing values in 'Load_consumed (kWh)' filled.
    """
    # Perform linear interpolation for missing values
    df["Load_consumed (kWh)"] = df["Load_consumed (kWh)"].interpolate(method='linear', limit_direction='both')

    # Round values to two decimal places
    df["Load_consumed (kWh)"] = df["Load_consumed (kWh)"].round(2)

    return df

**Function to fill missing time stamp**

In [36]:

def fill_missing_timestamp(df):
    # Reset index to make 'timestamp' a regular column
    df = df.reset_index()
    print(df.head())  # Debugging step: Print first few rows

    # Round the 'timestamp' to the nearest 15 minutes
    df['timestamp'] = pd.to_datetime(df['timestamp']).dt.round(freq='15min')

    # Remove duplicate timestamps (keeping the last one)
    df = df.loc[~df['timestamp'].duplicated(keep='last')]

    # Set 'timestamp' back as the index
    df.set_index('timestamp', inplace=True)

    # Calculate the start and end timestamps
    start = pd.to_datetime("2024-01-01")
    end = pd.to_datetime("2024-12-31")

    # Generate the desired date range with the specified frequency
    dates = pd.date_range(start=start, end=end, freq='15Min')

    # Reindex the dataframe to the new date range
    df = df.reindex(dates)
    df.index.name = 'timestamp'

    return df

**Calculating Load_consumed (kWh) row sum**



In [37]:
def df_total(df):
    #create pivot table
    df = pd.pivot_table(data=df,index=["Last Updated"],values=["Grid Consumption(kWh)"],columns="Meter")
    df.index = pd.to_datetime(df.index) #need this before sorting index
    df = df.sort_index(ascending=True)
    df = df.ffill()
    df["Total"] = df.sum(axis=1)
    df.drop(labels=["Grid Consumption(kWh)"],axis=1,inplace=True)
    df.reset_index(inplace=True) #Remove Index Name
    df.columns = [None] * len(df.columns) # Remove Column name

    df.columns = ['timestamp', 'Load_consumed (kWh)']
    df.set_index("timestamp",inplace=True)
    df["Load_consumed (kWh)"] = df["Load_consumed (kWh)"].round(2)



    #df.index.name = None
    #df.set_index("Last Updated",inplace=True)

    #df.rename_axis(None,inplace = True)
    #df.reset_index(inplace=True)
    #df.index.name = "timestamp"

    #df.drop(index=1, inplace=True) #Removes 2nd row index 1 is second row

    return df

**Function to Remove commas from Grid Consumption Column**

In [38]:
#Function to remove the comma from Grid Consumption Column
def replace_comma(df):
    df["Grid Consumption(kWh)"] = df["Grid Consumption(kWh)"].replace(",","",regex = True)
    df['Grid Consumption(kWh)'] = pd.to_numeric(df['Grid Consumption(kWh)'], errors='coerce')
    print("dtype of Grid Consumption(kWh)is:")
    print(df.dtypes)
    return(df)

```
def fetch_weather_data():
  weather_df = pd.read_csv("https://drive.google.com/uc?export=download&id=1mezuQQr_XKMYOT_X46UoUgbtDmoV_qPj")
  weather_df.drop(labels=["period", "dni", "ghi"], axis=1, inplace=True)
  return weather_df
```








```
def add_temperature(df, weather_df):
    # Convert 'period_end' in weather_df to datetime and set to Nepali time zone
    weather_df["period_end"] = pd.to_datetime(weather_df["period_end"], utc=True).dt.tz_convert("Asia/Kathmandu")

    # Remove timezone information
    weather_df["period_end"] = weather_df["period_end"].dt.tz_localize(None)

    # Set 'period_end' as the index
    weather_df.set_index("period_end", inplace=True)

    # Ensure 'timestamp' (or equivalent) in df is also datetime
    df.index = pd.to_datetime(df.index)

    # Resample weather_df to 15-minute intervals
    weather_df = weather_df.resample('15T').mean().round(2)

    # Outer join to align both DataFrames
    merged_df = df.join(weather_df, how='outer')

    # Drop rows with NaN values
    # merged_df.dropna(inplace=True)

    # Reset index to make 'period_end' a regular column
    merged_df.reset_index(inplace=True)
    merged_df.rename(columns={'index': 'period_end'}, inplace=True)

    # Add a holiday flag (1 if Saturday, 0 otherwise)
    merged_df['holiday'] = merged_df['period_end'].apply(lambda x: 1 if pd.Timestamp(x).weekday() == 5 else 0)

    # Set 'period_end' back as the index
    merged_df.set_index('period_end', inplace=True)

    return merged_df

```



**Function to fetch weather data stored in csv file**

In [39]:


def fetch_weather_data():
    """
    Fetches weather data, converts 'period_end' column from UTC to Nepali time (Asia/Kathmandu),
    and renames it to 'timestamp'.

    Returns:
    - pd.DataFrame: Processed DataFrame with the converted timestamp column.
    """
    # URL for the dataset
    url = "https://drive.google.com/uc?export=download&id=1mezuQQr_XKMYOT_X46UoUgbtDmoV_qPj"

    # Load the CSV into a DataFrame
    weather_df = pd.read_csv(url)

    # Drop unnecessary columns
    weather_df.drop(labels=["period", "dni", "ghi"], axis=1, inplace=True)

    # Convert 'period_end' to UTC datetime
    weather_df["period_end"] = pd.to_datetime(weather_df["period_end"], utc=True)

    # Convert to Nepali time zone (Asia/Kathmandu)
    weather_df["period_end"] = weather_df["period_end"].dt.tz_convert("Asia/Kathmandu")

    # Remove timezone information (optional)
    weather_df["period_end"] = weather_df["period_end"].dt.tz_localize(None)

    # Rename the column to 'timestamp'
    weather_df.rename(columns={"period_end": "timestamp"}, inplace=True)

    return weather_df

In [40]:
def add_temperature(df, weather_df):

    weather_df = weather_df.rename(columns={"period_end": "timestamp"})

    # Set 'period_end' as the index
    weather_df.set_index("timestamp", inplace=True)

    # Ensure 'timestamp' (or equivalent) in df is also datetime
    df.index = pd.to_datetime(df.index)

    # Resample weather_df to 15-minute intervals
    weather_df = weather_df.resample('15T').mean().round(2)

    # Outer join to align both DataFrames
    merged_df = df.join(weather_df, how='outer')

    # Drop rows with NaN values
    # merged_df.dropna(inplace=True)

    # Reset index to make 'period_end' a regular column
    merged_df.reset_index(inplace=True)
    merged_df.rename(columns={'index': 'timestamp'}, inplace=True)

    # Add a holiday flag (1 if Saturday, 0 otherwise)
    merged_df['holiday'] = merged_df['timestamp'].apply(lambda x: 1 if pd.Timestamp(x).weekday() == 5 else 0)

    # Set 'period_end' back as the index
    merged_df.set_index('timestamp', inplace=True)

    return merged_df


In [41]:
def add(df,weather_df):

  #print(df.head())
  #print(weather_df.head())
  df.reset_index(inplace=True)
  print(df.head())

  weather_df.set_index("timestamp",inplace=True)
  #print(weather_df.head())

  weather_df = weather_df.resample("15min").mean()
  #print(weather_df.head())

  weather_df.reset_index(inplace=True)
  print(weather_df.head())

  merged_df = df.merge(weather_df,how = 'outer',on = 'timestamp')
  merged_df['holiday'] = merged_df['timestamp'].apply(lambda x: 1 if pd.Timestamp(x).weekday() == 5 else 0)

  merged_df = merged_df[(merged_df['timestamp'] >= '2024-01-01') & (merged_df['timestamp'] < '2025-01-01')]
  merged_df.set_index("timestamp",inplace=True)
  print(merged_df.head())
  print(merged_df.tail())
  return(merged_df)



**Main Body**

In [42]:
#Uncomment for using all  consumer id from dictonary defined above
consumer_id = tuple(consumer.keys())
#Create a df for a single consumer only
#consumer_id = ["physics"] #
#print(consumer_id)
weather_df = fetch_weather_data()
temp_df = weather_df
print(weather_df.head())
for id in consumer_id:
    df = read_raw_csv(id)
    if df is not None:
        #print(f"first 5 rows of consumer {id} is: \n")
        #print(df.head())
        #print(df.dtypes)
        #print(df['Grid Consumption(kWh)'].unique())
        #sending data frame for pre-Processing
        df = data_preprocessing(df);
        #print(df.index)
        #print(df.head)


        #df = add_temperature(df,weather_df)
        df = add(df,weather_df)

        #print(df.head())
        df.to_csv("/content/drive/MyDrive/preprocessed_data/" + f"{id}_preprocessed.csv",float_format='%.2f');

        #call weather_df


        #print(f"first 5 rows of consumer {id} after pre-processing is: \n")
        #print(df.head())
        #print(df.dtypes)
        #print("After Processing")

    else:
        print(f"No data found for consumer {id}")
    weather_df = temp_df
    weather_df = weather_df.reset_index()

   air_temp           timestamp
0         8 2024-01-01 00:00:00
1         8 2024-01-01 00:05:00
2         8 2024-01-01 00:10:00
3         8 2024-01-01 00:15:00
4         8 2024-01-01 00:20:00


<ipython-input-32-be668fd1b39e>:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(Url,skiprows =1)


dtype of Grid Consumption(kWh)is:
Meter                     object
Grid Consumption(kWh)    float64
dtype: object
            timestamp  Load_consumed (kWh)
0 2024-01-07 12:00:26                 5.94
1 2024-01-07 12:05:19                 5.95
2 2024-01-07 12:11:36                 5.95
3 2024-01-07 12:16:39                 5.96
4 2024-01-07 12:20:33                 5.99
            timestamp  Load_consumed (kWh)
0 2024-01-01 00:15:00                  0.0
1 2024-01-01 00:30:00                  0.0
2 2024-01-01 00:45:00                  0.0
3 2024-01-01 01:00:00                  0.0
4 2024-01-01 01:15:00                  0.0
            timestamp  air_temp
0 2024-01-01 00:00:00       8.0
1 2024-01-01 00:15:00       8.0
2 2024-01-01 00:30:00       8.0
3 2024-01-01 00:45:00       8.0
4 2024-01-01 01:00:00       8.0
                     Load_consumed (kWh)  air_temp  holiday
timestamp                                                  
2024-01-01 00:00:00                  NaN       8.0        

<ipython-input-32-be668fd1b39e>:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(Url,skiprows =1)


dtype of Grid Consumption(kWh)is:
Meter                     object
Grid Consumption(kWh)    float64
dtype: object
            timestamp  Load_consumed (kWh)
0 2024-01-12 00:00:02                20.67
1 2024-01-12 00:05:53                20.69
2 2024-01-12 00:10:46                20.70
3 2024-01-12 00:15:39                20.72
4 2024-01-12 00:20:32                20.73
            timestamp  Load_consumed (kWh)
0 2024-01-01 00:15:00                  0.0
1 2024-01-01 00:30:00                  0.0
2 2024-01-01 00:45:00                  0.0
3 2024-01-01 01:00:00                  0.0
4 2024-01-01 01:15:00                  0.0
            timestamp  air_temp
0 2024-01-01 00:00:00       8.0
1 2024-01-01 00:15:00       8.0
2 2024-01-01 00:30:00       8.0
3 2024-01-01 00:45:00       8.0
4 2024-01-01 01:00:00       8.0
                     Load_consumed (kWh)  air_temp  holiday
timestamp                                                  
2024-01-01 00:00:00                  NaN       8.0        